# Playing Wordle with Information Theory

[Wordle](https://www.powerlanguage.co.uk/wordle/) is a popular browser game where each day you have to guess a 5-letter word. You get 6 guesses, each of which must be a 5-letter word. After each guess, the game highlights letters to show you how close you were. Each letter in your guess is:

* highlighted in green if it is in the answer and in the right place
* highlighted in yellow if it is in the answer but in the wrong place
* greyed out if it is not in the answer

For example, if you guessed SIREN and the answer was SOLAR, the output would be 🟩⬜🟨⬜⬜ - S is in the right place, R is in the word but in the wrong place, and the other letters aren't in the word.

The aim is to use this feedback to narrow it down to one word before you run out of guesses. 

What is the best starting word to use? Each guess gives the player information about the answer. Different words give different amounts of information. So the best starting word is the one that provides the most information about the answer. We can calculate the average amount of information returned for each guess using information theory. 

## Information Entropy

[Claude Shannon](https://en.wikipedia.org/wiki/Claude_Shannon) came up with a way to measure information in his 1948 paper [A Mathematical Theory of Communication](https://people.math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf). His [information entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) measure represents the average amount of information given by the outcome of a random variable. 

For a random variable $X$ with possible outcomes $x_1, ..., x_n$, the entropy $H(X)$ of $X$ is:

$$H(X) = -\sum_{i=1}^n P(X = x_i) \log P(X = x_i)$$

The smallest information entropy occurs when there is only one outcome, so $P(X = x_1) = 1$. Then $H(X) = -(1 \cdot \log 1) = 0$. 

The largest information entropy for a given number of outcomes $n$ occurs when $P(X = x_i) = \frac{1}{n}$ for all $i \in \{1, ..., n\}$.  Then:

$$H(X) = -\sum_{i = 1}^n \frac{1}{n} \log \frac{1}{n}$$

$$H(X) = \log{n}$$

Information entropy uses the natural logarithm (rather than base 2), so is measured in [nats](https://en.wikipedia.org/wiki/Nat_(unit)) rather than bits - 1 nat = $\frac{1}{\log 2} \approx$ 1.44 bits. 

### Information Entropy of Wordle

There are 2315 possible answers to Wordle puzzles. These are equally likely, so by the formula above the information entropy of the puzzle is log(2315) = 7.74 nats. 

Each guess returns five squares that are either green, yellow or grey/white - so there are 3^5 = 243 possible responses. If we found a guess that made these responses equally likely for each answer, the information entropy would be log(243) = 5.49 nats. This is the theoretical maximum amount of information from the first guess. How well can we do in practice?

## Methodology

This is an illustrative methodology for finding the most informative starting guess. Here I have used a list of 15 words for the possible answers and possible guesses, rather than the full lists (which take a very long time to calculate). 

### Playing Wordle in Python

wordle_guess is a function to check a guess word against an answer. The result is returned as a string of emoji.

In [1]:
import numpy as np
from emoji import emojize

square_emoji = [emojize(":white_large_square:"), emojize(":yellow_square:"), emojize(":green_square:")]

def wordle_guess(guess, answer):
    """Check guess against answer using rules of Wordle. Return a string of emoji."""
    guess = np.array(list(guess), dtype = str)
    answer = np.array(list(answer), dtype = str)
    res_numeric = np.isin(guess, answer).astype(int) + (guess == answer).astype(int)
    return ''.join([square_emoji[i] for i in res_numeric])

wordle_guess("siren", "solar")

'🟩⬜🟨⬜⬜'

Here is the response for our guess "siren" against each of the 15 sample words.

In [2]:
import pandas as pd
answers = ["siren", "solar", "smart", "scour", "pixel", "video", "burnt", "north", "sinew", "dirge", "focal", "adapt", "match", "outdo", "polka"]

df = pd.DataFrame({"guess": ["siren"] * len(answers), "answer": answers})
df["response"] = df.apply(lambda row: wordle_guess(row["guess"], row["answer"]), axis = 1)

df

,guess,answer,response
0,siren,siren,🟩🟩🟩🟩🟩
1,siren,solar,🟩⬜🟨⬜⬜
2,siren,smart,🟩⬜🟨⬜⬜
3,siren,scour,🟩⬜🟨⬜⬜
4,siren,pixel,⬜🟩⬜🟩⬜
5,siren,video,⬜🟩⬜🟩⬜
6,siren,burnt,⬜⬜🟩⬜🟨
7,siren,north,⬜⬜🟩⬜🟨
8,siren,sinew,🟩🟩⬜🟩🟨
9,siren,dirge,⬜🟩🟩🟨⬜


The responses are not unique. Let's count the number of possible answers given each response:

In [3]:
group_counts = df.groupby(["guess", "response"])["answer"].count()

group_counts

guess  response
siren  ⬜⬜⬜⬜⬜       5
       ⬜⬜🟩⬜🟨       2
       ⬜🟩⬜🟩⬜       2
       ⬜🟩🟩🟨⬜       1
       🟩⬜🟨⬜⬜       3
       🟩🟩⬜🟩🟨       1
       🟩🟩🟩🟩🟩       1
Name: answer, dtype: int64

Using scipy we can calculate the information entropy of this set of responses:

In [4]:
from scipy.stats import entropy
entropy(group_counts)

1.7670091910745693

The information entropy is 1.77. 

We can repeat this calculation for each initial guess to see which gives the greatest information entropy across our answer set:

In [44]:
from itertools import product

df = pd.DataFrame(product(answers, answers), columns = ["answer", "guess"])

df["response"] = df.apply(lambda row: wordle_guess(row["guess"], row["answer"]), axis=1)

group_counts = df.groupby(["guess", "response"], as_index = False).count()

entropy_df = group_counts.groupby('guess', as_index=False).agg({"answer": entropy}).rename(columns = {"answer": "entropy"})

entropy_df = entropy_df.sort_values("entropy", ascending = False).reset_index(drop = True)

entropy_df[:5]

,guess,entropy
0,north,2.615631
1,scour,2.430791
2,smart,2.430791
3,solar,2.430791
4,burnt,2.338372


For this set the best initial guess is "north", with an entropy of 2.62. This is close to the entropy of the 15-word set, which is log(15) = 2.71. 

## Results

Here are the results from running this algorithm over the whole set of Wordle guesses and answers:

In [5]:
result_df = pd.read_csv("top_words.csv")

result_df

,guess,entropy
0,soare,4.079837
1,roate,4.077632
2,raise,4.074257
3,raile,4.065800
4,reast,4.065625
5,slate,4.058914
6,crate,4.044426
7,salet,4.044224
8,irate,4.042016
9,trace,4.041428


In terms of information, the best word is "soare" (an archaic word for a young hawk), providing 4.080 nats of information. The best real word is "raise", which provides 4.074 nats. 